In [1]:
    try:
        from google.colab import drive

        print('Colab environment detected. Mounting drive...')
        drive.mount('/content/drive')

        print('Mounted. Switching to directory... ', end = '')
        %cd /content/drive/'My Drive'/CILroadseg
        print('done.')
    except:
        print('Colab environment not found. Working on ordinary directory.')

Colab environment detected. Mounting drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mounted. Switching to directory... /content/drive/My Drive/CILroadseg
done.


In [2]:
import numpy as np
np.random.seed(18)

import tensorflow as tf
tf.random.set_seed(33)

import sys
import os
import matplotlib.image as mpimg

from util.submit import *
from util.helpers import *
from util.visualize import *
from util.notebooks import *
from util.cross_validation import cross_validate

## Loading Training Data

In [3]:
train_dir = "training/images/"
gt_dir = "training/groundtruth/"
test_dir = "test/images/"

X, Y, X_test = nb_load_data(train_dir, gt_dir, test_dir)

Y = (Y >= 0.25) * 1
# This transformation is EXTREMELY important for the UNet Model

Loading training input...
Progress: done (100 images).
Loading training groundtruth...
Progress: done (100 images).
Loading test input...
Progress: done (94 images).

       Training data shape: (100, 400, 400, 3)
Training groundtruth shape: (100, 400, 400)
           Test data shape: (94, 608, 608, 3)


## Decomposer + ResNet/CNN

In [4]:
from discretize import *
from unet import *
from unet_framed import *

Using TensorFlow backend.


In [5]:
core_model = FramedUNetModel()

model = core_model
model_dsc = Discretizer(core_model)

In [ ]:
model_id = "unet8-brightness"

choice = 'tr'

if choice == 'cv':
  cross_validate(model, 4, X, Y)

elif choice == 'tr':
  model.initialize()
  model.train(Y, X) 

  nb_save_model(model, "saves/final/"+ model_id +".h5") 

else:
  model.initialize()
  model.load("saves/final/"+ model_id +".h5")

ft10
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 304, 304, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 304, 304, 32) 2432        input_1[0][0]                    
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 304, 304, 32) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 304, 304, 32) 25632       re_lu_1[0][0]                    
_______________________________________________________________________________________

## Submission Generation

In [ ]:
test_masks_dir = "test/pred/"
test_dir = "test/images/"
 
nb_predict_masks(model, test_dir, test_masks_dir + "unet/")

In [ ]:
image_paths = [test_masks_dir + "unet/" + file for file in os.listdir(test_masks_dir + "unet/")]
masks_to_submission("test/"+ model_id +".csv", image_paths)

In [ ]:
Xt = X_test[0:10]

Y_pred = model.classify(Xt)
Y_pred_dsc = model_dsc.classify(Xt)

view_image_array(Xt, Y_pred, Y_pred_dsc)